In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import telebot #Biblioteca de criação do Bot Telegram
from time import sleep 
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton #Biblioteca dos botões e markup
import pymysql
import getpass


senha = getpass.getpass('Digite a senha para conexão com o banco de dados:')

def checa_id(id_user0):
    global senha
    con = pymysql.connect(host='localhost', user='root', database='bot',password=senha, cursorclass=pymysql.cursors.DictCursor)
    c = con.cursor()
    sql = f'SELECT id_user FROM lojas WHERE id_user = {str(id_user0)}'
    c.execute(sql)
    resultado = c.fetchall() #Retorna todas os itens obtidas pela consulta na tabela
    c.close()
    con.close() # Desconectar do servidor
    return resultado
    
        
def uma_linha(nome_arquivo):
    with open (f'C:/Users/pdv/Desktop/gabriel/{nome_arquivo}', 'a+') as arquivo:
        for linha in arquivo:
            return linha        

            
chaveapi = uma_linha('novachave.txt')      

bot = telebot.TeleBot(chaveapi) #Cria uma instância do bot


def faz_botao(dicionario): #Função pra criar os botões de resposta
    markup = InlineKeyboardMarkup() #inicia o markup (marcação)
    for botao, retorno in dicionario.items(): #Desempacota os botões e seus retornos do dicionário
        markup.add(InlineKeyboardButton(str(botao), callback_data=str(retorno))) #cria o botão dentro do markup
    return markup #retorna o markup
    
    
def salva_banco(nome, id_, escolha):
    global senha
    resultado = checa_id(id_)
    if not id_ in resultado:
        con = pymysql.connect(host='localhost', user='root', database='bot',password=senha, cursorclass=pymysql.cursors.DictCursor) #Inicia a conexão com o BD
        c = con.cursor() #Instancia o cursor do db
        sql = f'INSERT INTO lojas (nome, id_user, {escolha}) VALUES (%s,%s,%s)'
        dados = (nome, id_, 1)
        c.execute(sql,dados)
        con.commit() #Lança as alterações no BD
        c.close() #Fecha cursor
        con.close() #Fecha conexão
    else:
        pass
    
@bot.message_handler(commands=["notas"]) #Decorador que cuida do comando /notas
def notas(mensagem):
    id_ = mensagem.chat.id #O id é uma parte da estrutura aninhada da mensagem ({'content_type': 'text', 'id': 9, 'message_id': 9,'from_user': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'username': 'Gbrdoidao', 'last_name': 'Doidao', 'language_code': 'pt-br', 'can_join_groups': None, 'can_read_all_group_messages': None, 'supports_inline_queries': None}, 'date': 1649940793,'chat': {'id': 945441763, 'type': 'private', 'title': None, 'username': 'Gbrdoidao', 'first_name': 'Gbr', 'last_name': 'Doidao', 'photo': None, 'bio': None, 'description': None, 'invite_link': None, 'pinned_message': None, 'permissions': None, 'slow_mode_delay': None, 'message_auto_delete_time': None, 'sticker_set_name': None, 'can_set_sticker_set': None, 'linked_chat_id': None, 'location': None},'sender_chat': None, 'forward_from': None, 'forward_from_chat': None, 'forward_from_message_id': None, 'forward_signature': None, 'forward_sender_name': None, 'forward_date': None, 'reply_to_message': None, 'via_bot': None, 'edit_date': None, 'media_group_id': None, 'author_signature': None, 'text': 'ao', 'entities': None, 'caption_entities': None, 'audio': None, 'document': None, 'photo': None, 'sticker': None, 'video': None, 'video_note': None, 'voice': None, 'caption': None, 'contact': None, 'location': None, 'venue': None, 'animation': None, 'dice': None, 'new_chat_member': None, 'new_chat_members': None, 'left_chat_member': None, 'new_chat_title': None, 'new_chat_photo': None, 'delete_chat_photo': None, 'group_chat_created': None, 'supergroup_chat_created': None, 'channel_chat_created': None, 'migrate_to_chat_id': None, 'migrate_from_chat_id': None, 'pinned_message': None, 'invoice': None, 'successful_payment': None, 'connected_website': None, 'reply_markup': None, 'json': {'message_id': 9, 'from': {'id': 945441763, 'is_bot': False, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'language_code': 'pt-br'}, 'chat': {'id': 945441763, 'first_name': 'Gbr', 'last_name': 'Doidao', 'username': 'Gbrdoidao', 'type': 'private'}, 'date': 1649940793, 'text': 'ao'}})
    fornecedor = mensagem.text[7:] #Pega a parte importante da mensagem (sem o /notas )
    print(mensagem.from_user.username,' pesquisou notas do ', fornecedor)
    dicio_pdf = {'Preço Notas':'preconf','Nota em PDF':'pdf'} #Botão pdf / preço {'nome_botão':'retorno do botão'}
    dicio_lojas = {'LJ1 - Guanabara' : 'lj1' ,'LJ2 - Primavera' : 'lj2','LJ3 - Iguatemi' : 'lj3','LJ4 - Flamboyant' : 'lj4', 'LJ5 - São Paulo' : 'lj5','LJ6 - Valinhos' : 'lj6',
                   'LJ7 - Ouro Verde' : 'lj7', 'ESCOLHIDO' : 'escolheu'} #Botões das lojas e seus retornos 
    bot.send_message(id_, ' Qual vc prefere? ' ,reply_markup=faz_botao(dicio_pdf)) #Cria o botão pro escolher entre PDF e meu preço
    bot.send_message(id_, ' LOJAS: ' ,reply_markup=faz_botao(dicio_lojas)) #Cria o botão pro escolher entre PDF e meu preço
    bot.reply_to(mensagem, 'pronto ;)') #Mensagem pra mostrar que deu certo
    

@bot.message_handler(func=lambda message: True) #Decorador que cuida de mensagens (não comandos)
def resposta(mensagem):
    print('falou cmg')
    texto = '''Olá! Seja bem vindo ao Fartura Bot  
    Estes são os meus Comandos:
    =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=''' #Mensagem padrão de resposta
    bot.reply_to(mensagem, texto) #Responde a mensagem dele com o texto padrão
    
    
@bot.callback_query_handler(func=lambda call: True)
def handle_query(chamado):
    nome = chamado.message.from_user.username
    id_ = chamado.message.chat.id
    retorno = chamado.data
    print(retorno)
    if retorno.startswith('lj'):
        print(f'loja {retorno} added')
        salva_banco(nome, id_, retorno)
        print(escolhas)
    elif retorno.startswith('escolheu'):
        print(escolhas)
#         lojas_escolhidas = []
    
print('passei pelo cod')    
while True:
    try:
        bot.polling(none_stop=True, interval=0, timeout=0) #Mantém o bot rodando infinitamente
    except:
        sleep(10) #se der erro ele espera 10 segundos


Digite a senha para conexão com o banco de dados:········
passei pelo cod


ERROR:root:No traceback has been produced, nothing to debug.


In [4]:
import pymysql
import getpass
from time import sleep


senha = getpass.getpass('Digite a senha para conexão com o banco de dados:')
con = pymysql.connect(host='localhost', user='root', database='bot',password=senha, cursorclass=pymysql.cursors.DictCursor)
# Preparar um cursor com o método .cursor()
c = con.cursor()
# sql = 'DROP TABLE lojas'
# c.execute(sql)

# sql = 'CREATE TABLE lojas (id INT AUTO_INCREMENT PRIMARY KEY, Nome VARCHAR(30) NOT NULL, id_user VARCHAR(30) NOT NULL, Lj1 BIT DEFAULT(0) NOT NULL, Lj2 BIT DEFAULT(0) NOT NULL, \
# Lj3 BIT DEFAULT(0) NOT NULL, Lj4 BIT DEFAULT(0) NOT NULL, Lj5 BIT DEFAULT(0) NOT NULL, Lj6 BIT DEFAULT(0) NOT NULL, Lj7 BIT DEFAULT(0) NOT NULL, Pdf BIT DEFAULT(0) NOT NULL)'
# c.execute(sql)
sql = f'INSERT INTO lojas (nome, id_user, Pdf) VALUES (%s,%s,%s)'
dados = ('gabriel', 1010192920102, 1)
c.execute(sql,dados)
con.commit()
print('ok')
c.close()
con.close()

Digite a senha para conexão com o banco de dados:········
ok
